1. Применить для тестовых вариантов и вариантов из репозиториев различные алгоритмы поиска ассоциативных правил при одинаковых начальных условиях (при одинаковых пороговых значениях для поддержки и достоверности) и сравнить полученные результаты. Для тестовых вариантов выполнить ручные расчеты (например, с применением MS Excel) и расчеты с применением программных библиотек на языке Python. Для вариантов из репозиториев выполнить расчеты с применением программных библиотек на языке Python. В качестве алгоритмов поиска ассоциативных правил использовать алгоритмы:
- Apriori (https://pypi.org/project/apriori-python/);
- Efficient Apriori (https://pypi.org/project/efficient-apriori/);
- FPGrowth (https://pypi.org/project/fpgrowth-py/).

In [ ]:
from apriori_python import apriori
transactions = list()  # TODO
freqItemSet, rules = apriori(transactions, minSup=0.5, minConf=0.5)
print(rules)

2. Сформировать базы ассоциативных правил с уровнем минимальной достоверности 60% и 80%. Вычислить для ассоциативных правил поддержку, достоверность, значимость.

3. Оценить время формирования искомых ассоциативных правил с применением различных алгоритмов и построить диаграммы, позволяющие выполнить сравнительный анализ.

4. Выполнить визуализацию ассоциативных правил (https://pypi.org/project/pyarmviz/).